In [1]:
import keras
from keras.layers import Input, Dense, Lambda, Flatten
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras.backend as K
import tensorflow as tf

from models import vgg
from utils.angles import deg2bit, bit2deg
from utils.losses import maad_from_deg 
from utils.losses import von_mises_log_likelihood_tf
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

Using TensorFlow backend.


In [2]:
xtr, ytr_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True)
image_height, image_width = xtr.shape[1], xtr.shape[2]
ytr_bit = deg2bit(ytr_deg)
yte_bit = deg2bit(yte_deg)

In [4]:
#x_tf = tf.placeholder(tf.float32, shape=[None, 3, image_width, image_height])
mu_pred_tf = tf.placeholder(tf.float32, shape=[None, 2])
kappa_tf = tf.placeholder(tf.float32, shape=[None, 1])
y_true_tf = tf.placeholder(tf.float32, shape=[None, 2])
y_pred_tf = tf.concat([mu_pred_tf, kappa_tf], axis=1)

In [29]:
def von_mises_neg_log_likelihood_keras(y_true, y_pred):
    mu_pred = y_pred[:,0:2]
    #kappa_pred = tf.abs(y_pred[:, 2:])
    kappa_pred = tf.ones(shape=tf.shape(y_pred[:, 2:]))
    return -von_mises_log_likelihood_tf(y_true, mu_pred, kappa_pred, input_type='biternion')

In [30]:
X = Input(shape=[50,50,3])
vgg_x = vgg.vgg_model(final_layer=False,
                      image_height=50,
                      image_width=50)(X)

mu = Lambda(lambda x: K.l2_normalize(x, axis=1))(Dense(2)(vgg_x))
kappa = Dense(1)(vgg_x) # Input(shape=[1]) 
mu_kappa = concatenate([mu, kappa])

biternion_vgg_prob = Model(X, mu_kappa)

optimizer = keras.optimizers.Adam(lr=0.0001)

biternion_vgg_prob.compile(loss=von_mises_neg_log_likelihood_keras,
                           optimizer='adam')

In [ ]:
biternion_vgg_prob.fit(x=xtr , y=ytr_bit, batch_size=100, validation_split=0.1, epochs=5)

Train on 7002 samples, validate on 778 samples
Epoch 1/5
6900/7002 [============================>.] - ETA: 0s - loss: 1.3314

In [23]:
mu_kappa = biternion_vgg_prob.predict(xte)
yte_preds = bit2deg(mu_kappa[:,0:2])
kappa_te_preds = mu_kappa[:,2]

In [26]:
np.abs(kappa_te_preds)

array([ 0.95914418,  1.16644001,  1.21386981,  1.18979573,  0.77466136,
        0.87817949,  0.81823808,  0.812769  ,  0.75922096,  1.11363792,
        1.16595829,  0.8266753 ,  0.67536193,  1.26709664,  1.18848157,
        1.17751908,  1.07461357,  1.08187842,  1.23312712,  1.32924855,
        1.28198802,  1.04357648,  0.84404445,  0.84525782,  1.05876946,
        0.99963069,  1.49382162,  0.97151214,  0.99819058,  0.96586686,
        1.48662722,  1.38902879,  1.43601644,  1.49899542,  1.37686074,
        1.07814431,  0.77883059,  0.93204808,  1.0832001 ,  1.36207318,
        0.89538097,  0.63032961,  0.68787169,  0.77558833,  0.75076693,
        1.07486343,  0.9722659 ,  1.0286864 ,  1.00117862,  1.14411855,
        1.2603631 ,  0.97109264,  1.12154925,  0.93705302,  0.9458105 ,
        0.92718327,  0.73657447,  1.18889332,  0.79628795,  0.816742  ,
        0.9899922 ,  0.85392892,  0.83776528,  0.71300447,  0.88809937,
        1.54823232,  1.36320078,  1.13305974,  0.77141607,  1.61

In [25]:
loss = maad_from_deg(yte_preds, yte_deg)

mean_loss = np.mean(loss)
std_loss = np.std(loss)

print("MAAD error (test) : %f ± %f" % (mean_loss, std_loss))

MAAD error (test) : 57.973165 ± 46.657730
